In [3]:
#!pip install beautifulsoup4 requests selenium

In [4]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import re

Collecting Signal IDs

Viewing IDs List

In [5]:
IDs = pd.read_csv("signal_list.csv")
IDs.describe()


,Signal Id
count,2.091000e+03
mean,1.756951e+06
std,2.352849e+05
min,8.527800e+04
25%,1.733086e+06
50%,1.838356e+06
75%,1.903918e+06
max,1.943806e+06


View Signal Information

In [6]:
#signals = pd.read_csv("signal_details.csv")
#signals.describe()

In [9]:

def load_proxies_from_csv(file_path):
    proxies = []
    with open(file_path, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        for row in csv_reader:
            proxies.append(row)  # Assuming the proxy address is in the first column
    return proxies

def make_request(url, proxy):
    try:
        response = requests.get(url, proxies={'http': proxy, 'https': proxy})
        if response.status_code == 200:
            return response
        elif response.status_code == 403:
            print(f"403 Forbidden error with proxy: {proxy}. Switching proxy...")
            return None
        else:
            print(f"Error {response.status_code} occurred with proxy: {proxy}")
            return None
    except requests.RequestException as e:
        print(f"An error occurred with proxy: {proxy}. Error: {str(e)}")
        return None

def crawl_with_proxies(url, proxy_list):
    for proxy in proxy_list:
        print(f"Trying proxy: {proxy}")
        response = make_request(url, proxy)
        if response is not None:
            # Process the response or perform desired actions
            print(response)
            break  # Exit the loop if a successful response is obtained

# Usage example
csv_file_path = 'proxies_list.csv'
proxies = load_proxies_from_csv(csv_file_path)
proxies[0]
response = requests.get("https://www.mql5.com/en/signals/1491698?source=Site+Signals+MT5+Tile#!tab=stats", proxies= {'http': proxies[0], 'https': proxy = proxies[0]}

['14.139.242.7:80']

In [8]:
base_url_singleID = 'https://www.mql5.com/en/signals/'
with open('signal_details.csv', 'w', newline='') as signal_detailsFile:
  writer = csv.writer(signal_detailsFile)
  writer.writerow(['Signal Id','Trades','ProfitTrades','LossTrades','BestTrades','WorstTrades','GrossProfit','GrossLoss','SharpeRatio','RecoveryFactor','ProfitFactor','ExpectedPayoff','AlgoTrading'])
#load the csv containing Signal IDs
for proxy in proxies:
  start_point = 782
  for row in IDs.iloc[start_point:]['Signal Id']:
    indexing = start_point + 1
    url = base_url_singleID + str(row) + "?source=Unknown#!tab=stats"
    #Crawl each signal
    response = make_request(url,proxies[0])
    print(response.status_code)
    if response.status_code == 403:
      start_point = indexing
      break
    soup = BeautifulSoup(response.content, 'html.parser')
    
    values = soup.find_all('div', class_='s-data-columns__value')
    if len(values) == 0:
        continue
    # Loop over each product and extract the data
    signalTrades = values[0].text 


  # Get total number of profit trades
    s = values[1].text
    number_regex = r"([\d\s]+)\s*\(.*\)"
    match = re.search(number_regex, s)
    if match:
        number = match.group(1)
        # Remove spaces and normalize the number
        normalized_number = float(number.replace(" ", "").replace(",", ""))
    signalProfitTrades = normalized_number

    # Get total number of loss trades

    s = values[2].text
    number_regex = r"([\d\s]+)\s*\(.*\)"
    match = re.search(number_regex, s)
    if match:
        number = match.group(1)
        # Remove spaces and normalize the number
        normalized_number = float(number.replace(" ", "").replace(",", ""))
    signalLossTrades = normalized_number

    signalBestTrade = values[3].find('i').text
    signalWorstTrade = values[4].find('i').text

    signalGrossProfit = values[5].find('i').text
    signalGrossLoss = values[6].find('i').text



    signalSharpeRatio = values[9].text

    signalRecoveryFactor = values[15].text


    signalProfitFactor = values[18].text

    signalExpectedPayoff = values[19].find('i').text

    # Find the <svg> element

    if "Algo trading" in values[25].parent.text:
      pattern = r'%'
      signalAlgoTrading = re.sub(pattern, '', values[25].text)
    else :
      pattern = r'%'
      signalAlgoTrading = re.sub(pattern, '', values[26].text)

    values2 = soup.find_all('div', class_='s-list-info__value')
    signalGrowth = re.sub(pattern, '', values2[0].text) #percentage
    with open('signal_details.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([row,signalTrades,signalProfitTrades,signalLossTrades,signalBestTrade,signalWorstTrade,signalGrossProfit,signalGrossLoss,signalSharpeRatio,signalRecoveryFactor,signalProfitFactor,signalExpectedPayoff,signalAlgoTrading])

TypeError: expected string or bytes-like object, got 'list'